In [1]:
import pandas as pd
import ast

In [ ]:
df_stops_total = pd.read_csv('../../final-tables/stations.csv')
df_1951 = pd.read_csv('../initial/stops_df_1951-initial.csv')

In [3]:
df_1951.head(3)

,Unnamed: 0,stop_name,line_count,type,in_lines
0,0,Hallesches Tor (Stresemannstrasse),1,strassenbahn,['1']
1,1,Prinzenstrasse,3,strassenbahn,['1']
2,2,Prinzenstrasse,3,u-bahn,"['B I', 'B II']"


In [4]:
df_1951.drop(columns=['Unnamed: 0'], inplace=True)
df_1951["location"] = ""
df_1951["identifier"] = ""
df_1951["stop_id_other"] = ""
df_1951["previous_in_lines"] = ""
df_1951["stop_description"] = ""

In [5]:
df_stops_total.tail(3)

,stop_id,stop_name,type,location,in_lines,identifier,stop_description
5484,19641453,Zum Heckeshorn,bus,"52.43307505069834, 13.164008736441371",A51,NaN,Zum Heckeshorn Ecke Am Großen Wannsee
5485,19641454,Zweibrücker Strasse,bus,"52.54677409100436, 13.188764018875105",A63,NaN,NaN
5486,19641455,Zühlsdorf,bus,"52.7162250652868, 13.366925671584623",AS,NaN,NaN


In [6]:
# Function to convert string representation of set to list
def convert_to_list(value):
    try:
        # Use ast.literal_eval to safely convert string to set
        items = ast.literal_eval(value)
        
        # Check if the result is a set, and convert it to a list
        if isinstance(items, set):
            return list(items)
        # Check if the result is an integer, and convert it to a list with a single element
        elif isinstance(items, int):
            return [items]
        # For other types, return a list with the original value
        else:
            return [value]
    except (SyntaxError, ValueError):
        # Handle the case where literal_eval fails (e.g., invalid string)
        # Try to convert the value to an integer, and return a list with a single element if successful
        try:
            return [int(value)]
        except ValueError:
            return [value]

# Apply the function to the 'in_lines' column
df_stops_total['in_lines'] = df_stops_total['in_lines'].apply(convert_to_list)
df_1951['in_lines'] = df_1951['in_lines'].apply(convert_to_list)

In [7]:
df_stops_total['in_lines'] = df_stops_total['in_lines'].apply(lambda x: ','.join(map(str, x)))
df_1951['in_lines'] = df_1951['in_lines'].apply(lambda x: ','.join(map(str, x)))

In [8]:
df_stops_total.head(3)

,stop_id,stop_name,type,location,in_lines,identifier,stop_description
0,19560,Adamstrasse Ecke Wilhelmstrasse,bus,"52.5210974258045, 13.187990320264987",['A13'],NaN,NaN
1,19561,Adlershof,s-bahn,"52.434722222222,13.541388888889","['KBS 100a', 'KBS 103']",Q323551,NaN
2,19562,Adolf-Scheidt-Platz,strassenbahn,"52.478194856980814, 13.37836902238447",['95'],NaN,NaN


I have two dataframes df_long and df_short, I want to add the information from df_long columns combined_location and combined_identifier to df_short if there is a match between the original_name column from df_long with the stop_name column of df_short. I have to create combined_location and combined_identifier columns in df_short first

In [9]:
for index, row in df_1951.iterrows():
    # Check if row["stop_name"] is in df_1960["original_name"]
    if row["stop_name"] in df_stops_total["stop_name"].values:
        # Find matching rows in df_1960
        matching_rows = df_stops_total[df_stops_total["stop_name"] == row["stop_name"]]
        
        # Check if those two rows have matching row["type"] values
        matching_rows = matching_rows[matching_rows["type"] == row["type"]]
        
        # Check if there are any matching rows
        if not matching_rows.empty:
            df_1951.at[index, 'location'] = '+ '.join(matching_rows['location'].astype(str).values)
            df_1951.at[index, 'identifier'] = '+ '.join(matching_rows['identifier'].astype(str).values)
            df_1951.at[index, 'stop_description'] = '+ '.join(matching_rows['stop_description'].astype(str).values)
            df_1951.at[index, 'stop_id_other'] = '+ '.join(matching_rows['stop_id'].astype(str).values)
            df_1951.at[index, 'previous_in_lines'] = '+ '.join(matching_rows['in_lines'].astype(str).values)
        else:
            # Handle the case where there are no matching rows in df_1960
            df_1951.at[index, 'location'] = ''
            df_1951.at[index, 'identifier'] = ''
            df_1951.at[index, 'stop_description'] = ''
            df_1951.at[index, 'stop_id_other'] = ''
            df_1951.at[index, 'previous_in_lines'] = ''




        # # Convert values to strings and replace NaN with empty strings
        # combined_location = " + ".join(matching_rows["combined_location"].astype(str).fillna(''))
        # combined_identifier = " + ".join(matching_rows["combined_identifier"].astype(str).fillna(''))
            
        # # Update row["location"] and row["identifier"]
        # row["location"] += " + " + combined_location
        # row["identifier"] += " + " + combined_identifier

# Now, df_1961 should have updated "location" and "identifier" columns based on the conditions.


In [10]:
# Function to split string values containing '+' and convert to a unique string
def split_values(cell_value):
    unique_values = sorted(set(element.strip() for element in cell_value.split('+')))
    return '+'.join(unique_values)

# Apply the function to each cell in the specified columns
df_1951['location'] = df_1951['location'].apply(split_values)
df_1951['identifier'] = df_1951['identifier'].apply(split_values)

In [11]:
df_1951.drop(columns=['line_count', "stop_id_other"], inplace=True)

In [12]:
# Function to concatenate unique values in other columns
def concat_unique_values(series):
    unique_values = set(series)
    return '+ '.join(unique_values)

# Group by specified columns and aggregate the other columns
df_1951 = df_1951.groupby(['stop_name', 'type', 'in_lines', 'location']).agg({
    'identifier': concat_unique_values,
    'previous_in_lines': concat_unique_values,
    'stop_description': concat_unique_values
}).reset_index()

In [13]:
df_1951.to_csv('1951-stops-combined.csv')

In [14]:
df_1951.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 957 entries, 0 to 956
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   stop_name          957 non-null    object
 1   type               957 non-null    object
 2   in_lines           957 non-null    object
 3   location           957 non-null    object
 4   identifier         957 non-null    object
 5   previous_in_lines  957 non-null    object
 6   stop_description   957 non-null    object
dtypes: object(7)
memory usage: 52.5+ KB


In [15]:
df_1951.head(10)

,stop_name,type,in_lines,location,identifier,previous_in_lines,stop_description
0,Adamstrasse Ecke Wilhelmstrasse,bus,['A13'],"52.5210974258045, 13.187990320264987",nan,['A13'],nan
1,Adlershof,s-bahn,"['KBS 100a', 'KBS 103']","52.434722222222,13.541388888889",Q323551,"['KBS 100a', 'KBS 103']+ KBS 103,KBS 100a+ KBS...",nan+ Berlin-Adlershof station+ Berlin-Adlersho...
2,"Adlershof, Oppenbrücke (Benzolwerk)",strassenbahn,['84'],,,,
3,"Ahrensfelde, Lindenbergerstrasse (Kirche)",bus,['A39'],,,,
4,Alboinstrasse Ecke Schönebergerstrasse,strassenbahn,['6'],"52.4698434808757, 13.372061552266187",nan,['6'],nan
5,Albrechtshof,s-bahn,['KBS 103'],"52.5494,13.1283",Q800470,['KBS 103']+ KBS 103+ KBS 103+ KBS 103,nan+ Berlin Albrechtshof station+ Berlin Albre...
6,Alexanderplatz,s-bahn,"['KBS 101', 'KBS 103', 'KBS 102']","52.521388888889,13.411944444444",Q111324700,"['KBS 102', 'KBS 101', 'KBS 103']+ KBS 102,KBS...",nan+ S-Bahnhof Berlin Alexanderplatz+ S-Bahnho...
7,Alexanderplatz,strassenbahn,['1'],,,,
8,Alexanderplatz,u-bahn,"['A I', 'D', 'E']","52.5214,13.4119",Q17173458,"['D', 'E', 'A I']+ E,D,A I,A II+ A,E+ A,E",nan+ Alexanderplatz u-bahn station+ Alexanderp...
9,"Alexanderplatz, Grunerstrasse",strassenbahn,"['73', '71', '72']","52.52008326268696, 13.41408418961877",nan,"['74', '22', '71', '72']+ 82,74,74E,72,71+ 71+ 71","nan+ Alexanderplatz, Grunerstrasse+ Alexanderp..."
